In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, ZeroPadding2D, BatchNormalization
from keras import regularizers
from keras.optimizers import SGD
from keras import losses
from keras import utils
from keras import callbacks
import keras
import h5py 

Using TensorFlow backend.


In [2]:
import numpy as np
from PIL import Image
import copy

# constants
CLASS = 200
SAMPLES_PER_CLASS = 500
TOTAL_SAMPLES = CLASS * SAMPLES_PER_CLASS
COLOR_CHANNELS = 3
IMAGE_WIDTH = 64
IMAGE_HEIGHT = 64
VAL_SAMPLES = 10000

# read all of the word net id
wnids = [id.strip('\n') for id in open('/datasets/tmp/cg181fdn/tiny-imagenet-200/wnids.txt').readlines()]

# data will store all of the data
data = {}

# train data
data['train'] = {}
data['train']['data'] = np.ndarray(shape=(TOTAL_SAMPLES, IMAGE_WIDTH, IMAGE_HEIGHT, COLOR_CHANNELS), dtype=np.uint8)
data['train']['target'] = np.ndarray(shape=(TOTAL_SAMPLES,), dtype=np.uint8)

# validation data
data['val'] = {}
data['val']['data'] = np.ndarray(shape=(VAL_SAMPLES, IMAGE_WIDTH, IMAGE_HEIGHT, COLOR_CHANNELS), dtype=np.uint8)
data['val']['target'] = np.ndarray(shape=(VAL_SAMPLES,), dtype=np.uint8)

# iterate through work net ids
print("storing training:")
for i in range(len(wnids)):
    wnid = wnids[i]
    print("%s: %d / %d" % (wnid, i + 1, len(wnids)))
    for j in range(500):
        temp = []
        path = "/datasets/tmp/cg181fdn/tiny-imagenet-200/train/{0}/images/{0}_{1}.JPEG".format(wnid, j)
        data['train']['data'][i * SAMPLES_PER_CLASS + j] = np.array(Image.open(path).convert('RGB'))
        data['train']['target'][i * SAMPLES_PER_CLASS + j] = wnids.index(wnid)

# get the validation data
print("storing validation:")
for i, line in enumerate(map(lambda s: s.strip(), open('/datasets/tmp/cg181fdn/tiny-imagenet-200/val/val_annotations.txt'))):
    name, wnid = line.split('\t')[0:2]
    temp = []
    path = "/datasets/tmp/cg181fdn/tiny-imagenet-200/val/images/{0}".format(name)
    data['val']['data'][i] = np.array(Image.open(path).convert('RGB'))
    data['val']['target'][i] = wnids.index(wnid)

storing training:
n02124075: 1 / 200
n04067472: 2 / 200
n04540053: 3 / 200
n04099969: 4 / 200
n07749582: 5 / 200
n01641577: 6 / 200
n02802426: 7 / 200
n09246464: 8 / 200
n07920052: 9 / 200
n03970156: 10 / 200
n03891332: 11 / 200
n02106662: 12 / 200
n03201208: 13 / 200
n02279972: 14 / 200
n02132136: 15 / 200
n04146614: 16 / 200
n07873807: 17 / 200
n02364673: 18 / 200
n04507155: 19 / 200
n03854065: 20 / 200
n03838899: 21 / 200
n03733131: 22 / 200
n01443537: 23 / 200
n07875152: 24 / 200
n03544143: 25 / 200
n09428293: 26 / 200
n03085013: 27 / 200
n02437312: 28 / 200
n07614500: 29 / 200
n03804744: 30 / 200
n04265275: 31 / 200
n02963159: 32 / 200
n02486410: 33 / 200
n01944390: 34 / 200
n09256479: 35 / 200
n02058221: 36 / 200
n04275548: 37 / 200
n02321529: 38 / 200
n02769748: 39 / 200
n02099712: 40 / 200
n07695742: 41 / 200
n02056570: 42 / 200
n02281406: 43 / 200
n01774750: 44 / 200
n02509815: 45 / 200
n03983396: 46 / 200
n07753592: 47 / 200
n04254777: 48 / 200
n02233338: 49 / 200
n04008634: 

In [3]:
class TestCallback(callbacks.Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [ ]:
def vgg_like():
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64,64,3)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(256, activation='relu',
                    kernel_regularizer=regularizers.l1_l2(l1=1e-6,l2=1e-6)))
    model.add(Dropout(0.5))
    model.add(Dense(200, activation='softmax',
                   kernel_regularizer=regularizers.l1_l2(l1=1e-5,l2=1e-5)))
    
    return model

In [4]:
def vgg_16():    
    model = Sequential()

    model.add(ZeroPadding2D(padding=(1,1), input_shape=(64,64,3)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', name="conv1_1"))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', name="conv1_2"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), name="block1_pool"))

    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', name="conv2_1"))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', name="conv2_2"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), name="block2_pool"))

    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', name="conv3_1"))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', name="conv3_2"))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', name="conv3_3"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), name="block3_pool"))

    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', name="conv4_1"))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', name="conv4_2"))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', name="conv4_3"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), name="block4_pool"))

    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu',
                     kernel_regularizer=regularizers.l1_l2(l1=1e-7,l2=1e-7), name="conv5_1"))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu',
                     kernel_regularizer=regularizers.l1_l2(l1=1e-6,l2=1e-6), name="conv5_2"))
    model.add(ZeroPadding2D(padding=(1,1)))
    model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu',
                     kernel_regularizer=regularizers.l1_l2(l1=1e-5,l2=1e-5), name="conv5_3"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2), name="block5_pool"))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu',
                    kernel_regularizer=regularizers.l1_l2(l1=1e-4,l2=1e-4), name="fc6"))
    model.add(Dropout(0.75, name="drop6"))
    model.add(Dense(4096, activation='relu',
                    kernel_regularizer=regularizers.l1_l2(l1=1e-4,l2=1e-4), name="fc7"))
    model.add(Dropout(0.75, name="drop7"))
    model.add(Dense(200, activation='softmax', name="fc8"))
    
    return model

In [ ]:
def alexnet():
    model = Sequential()

    model.add(Conv2D(filters=96, kernel_size=(7, 7), activation='relu', input_shape=(64,64,3)))    
    model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(1,1)))

    model.add(Conv2D(filters=128, kernel_size=(7, 7), activation='relu'))    
    model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(1,1)))

    model.add(Conv2D(filters=192, kernel_size=(3, 3), activation='relu'))    
    model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(1,1)))
    
    model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu'))    
    model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(1,1)))
    
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
    model.add(Dense(4096, activation='relu'))
    model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))
    model.add(Dense(200, activation='softmax', name="fc8"))
    model.add(BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9))

    return model

In [9]:
shuffle_train_index = np.random.permutation(len(data['train']['data']))
training_images = np.array(data['train']['data'])[shuffle_train_index]
training_labels = np.array(data['train']['target'])[shuffle_train_index]
training_labels = utils.to_categorical(training_labels, num_classes=200)

val_images = np.array(data['val']['data'])
val_labels = np.array(data['val']['target'])
val_labels = utils.to_categorical(val_labels, num_classes=200)

myModel = vgg_16()
#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
op = keras.optimizers.Adadelta(lr=0.01, rho=0.95, epsilon=1e-08, decay=0.0)
myModel.compile(loss='categorical_crossentropy', optimizer=op,
                metrics=['accuracy'])
hist = myModel.fit(x=training_images, y=training_labels, batch_size=128,
                 epochs=100, callbacks=[TestCallback((val_images, val_labels))])

Epoch 1/100
 99968/100000 [============================>.] - ETA: 0s - loss: 41.2651 - acc: 0.0049
Testing loss: 40.3089189026, acc: 0.0059

100000/100000 [==============================] - 138s - loss: 41.2648 - acc: 0.0049   
Epoch 2/100
 99968/100000 [============================>.] - ETA: 0s - loss: 39.3523 - acc: 0.0050
Testing loss: 38.3923794312, acc: 0.0069

100000/100000 [==============================] - 128s - loss: 39.3520 - acc: 0.0050   
Epoch 3/100
 99968/100000 [============================>.] - ETA: 0s - loss: 37.4575 - acc: 0.0056
Testing loss: 36.525319104, acc: 0.0104

100000/100000 [==============================] - 128s - loss: 37.4572 - acc: 0.0056   
Epoch 4/100
 99968/100000 [============================>.] - ETA: 0s - loss: 35.6528 - acc: 0.0066
Testing loss: 34.7930457764, acc: 0.0139

100000/100000 [==============================] - 128s - loss: 35.6525 - acc: 0.0066   
Epoch 5/100
 99968/100000 [============================>.] - ETA: 0s - loss: 34.0428 - ac

100000/100000 [==============================] - 128s - loss: 15.3278 - acc: 0.0763   
Epoch 37/100
 99968/100000 [============================>.] - ETA: 0s - loss: 15.1181 - acc: 0.0799
Testing loss: 14.888698175, acc: 0.1064

100000/100000 [==============================] - 129s - loss: 15.1182 - acc: 0.0799   
Epoch 38/100
 99968/100000 [============================>.] - ETA: 0s - loss: 14.9137 - acc: 0.0808
Testing loss: 14.665787619, acc: 0.1109

100000/100000 [==============================] - 128s - loss: 14.9137 - acc: 0.0808   
Epoch 39/100
 39552/100000 [==========>...................] - ETA: 74s - loss: 14.7733 - acc: 0.0836

KeyboardInterrupt: 

In [ ]:
for p in pretrain:
    print(p)